2602141871 - Kayla Masayuningtyas - LB09

**1. import library**

In [2]:
# %pip install xgboost

In [3]:
# import library

import pandas as pd
import numpy as np
import seaborn as sn
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split, GridSearchCV

**2. Load data**

In [4]:
df = pd.read_csv('dataset.csv')
df

,Unnamed: 0,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,churn
0,0,37969,15780804,Ugorji,850.0,France,Male,42.0,5,0.00,2,1,1,105560.01,0
1,1,121427,15593974,Dickson,597.0,Germany,Male,46.0,9,112291.06,2,1,1,47410.06,0
2,2,151801,15655274,Teng,637.0,Germany,Male,35.0,3,145974.79,2,1,0,180934.67,0
3,3,149981,15645404,McWilliams,695.0,France,Male,39.0,8,0.00,1,1,0,162669.73,0
4,4,115198,15625881,Yobachi,695.0,France,Female,29.0,0,109109.89,1,1,0,120770.80,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41254,41254,106740,15662289,Hsu,562.0,France,Female,35.0,9,0.00,2,1,1,187926.11,0
41255,41255,38403,15782530,Hsing,653.0,Spain,Male,39.0,3,0.00,1,1,0,131083.99,0
41256,41256,160654,15739692,Cisneros,526.0,France,Female,37.0,9,0.00,2,1,0,153383.54,0
41257,41257,44993,15780386,Lucchese,571.0,France,Male,40.0,2,0.00,2,1,1,123866.43,0


**3. data observation**

In [5]:
df.shape

(41259, 15)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41259 entries, 0 to 41258
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       41259 non-null  int64  
 1   id               41259 non-null  int64  
 2   CustomerId       41259 non-null  int64  
 3   Surname          41259 non-null  object 
 4   CreditScore      41251 non-null  float64
 5   Geography        41259 non-null  object 
 6   Gender           41259 non-null  object 
 7   Age              41259 non-null  float64
 8   Tenure           41259 non-null  int64  
 9   Balance          41259 non-null  float64
 10  NumOfProducts    41259 non-null  int64  
 11  HasCrCard        41259 non-null  int64  
 12  IsActiveMember   41259 non-null  int64  
 13  EstimatedSalary  41259 non-null  float64
 14  churn            41259 non-null  int64  
dtypes: float64(4), int64(8), object(3)
memory usage: 4.7+ MB


In [7]:
df.head()

,Unnamed: 0,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,churn
0,0,37969,15780804,Ugorji,850.0,France,Male,42.0,5,0.00,2,1,1,105560.01,0
1,1,121427,15593974,Dickson,597.0,Germany,Male,46.0,9,112291.06,2,1,1,47410.06,0
2,2,151801,15655274,Teng,637.0,Germany,Male,35.0,3,145974.79,2,1,0,180934.67,0
3,3,149981,15645404,McWilliams,695.0,France,Male,39.0,8,0.00,1,1,0,162669.73,0
4,4,115198,15625881,Yobachi,695.0,France,Female,29.0,0,109109.89,1,1,0,120770.80,0


In [8]:
# remove variabel yang tidak akan di gunakan
df = df.drop(columns=['id', 'CustomerId', 'Surname'])
df.head()

,Unnamed: 0,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,churn
0,0,850.0,France,Male,42.0,5,0.00,2,1,1,105560.01,0
1,1,597.0,Germany,Male,46.0,9,112291.06,2,1,1,47410.06,0
2,2,637.0,Germany,Male,35.0,3,145974.79,2,1,0,180934.67,0
3,3,695.0,France,Male,39.0,8,0.00,1,1,0,162669.73,0
4,4,695.0,France,Female,29.0,0,109109.89,1,1,0,120770.80,0


In [9]:
df['churn'].value_counts()

0    32518
1     8741
Name: churn, dtype: int64

In [10]:
missing_data = df.isnull().sum()
duplicated_data = df.duplicated().sum()

missing_duplicated_info = pd.DataFrame({
    'Missing Data': missing_data,
    'Duplicated Data': duplicated_data
})

missing_duplicated_info

,Missing Data,Duplicated Data
Unnamed: 0,0,0
CreditScore,8,0
Geography,0,0
Gender,0,0
Age,0,0
Tenure,0,0
Balance,0,0
NumOfProducts,0,0
HasCrCard,0,0
IsActiveMember,0,0


In [28]:
df.describe()

,Unnamed: 0,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,churn
count,41259.000000,41251.000000,41259.000000,41259.000000,41259.000000,41259.000000,41259.000000,41259.000000,41259.000000,41259.000000
mean,20629.000000,656.576253,38.164570,5.025570,55045.726741,1.552655,0.753581,0.496619,112741.311618,0.211857
std,11910.591715,80.196755,8.893375,2.805195,62703.556191,0.549379,0.430931,0.499995,50460.520109,0.408629
min,0.000000,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,10314.500000,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74800.990000,0.000000
50%,20629.000000,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,118913.530000,0.000000
75%,30943.500000,710.000000,42.000000,7.000000,119639.315000,2.000000,1.000000,1.000000,155846.700000,0.000000
max,41258.000000,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


**4. splitign data**

In [12]:
input_df = df.drop('churn', axis=1)
output_df = df['churn']

x_train, x_test, y_train, y_test = train_test_split(input_df, output_df, test_size=0.2, random_state=42)

In [13]:
x_train.shape

(33007, 11)

In [14]:
x_test.shape

(8252, 11)

**5. missing value handling**

In [15]:
np.mean(x_train['CreditScore'])

656.9331818181818

In [16]:
x_train['CreditScore'].fillna(656.9, inplace = True)
x_test['CreditScore'].fillna(656.9, inplace = True)

In [17]:
x_train.isnull().sum()

Unnamed: 0         0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

**6. Feature Encoding**

In [18]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33007 entries, 23629 to 15795
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       33007 non-null  int64  
 1   CreditScore      33007 non-null  float64
 2   Geography        33007 non-null  object 
 3   Gender           33007 non-null  object 
 4   Age              33007 non-null  float64
 5   Tenure           33007 non-null  int64  
 6   Balance          33007 non-null  float64
 7   NumOfProducts    33007 non-null  int64  
 8   HasCrCard        33007 non-null  int64  
 9   IsActiveMember   33007 non-null  int64  
 10  EstimatedSalary  33007 non-null  float64
dtypes: float64(4), int64(5), object(2)
memory usage: 3.0+ MB


In [19]:
df['Geography'].value_counts()

France     23708
Spain       8961
Germany     8590
Name: Geography, dtype: int64

In [20]:
df['Gender'].value_counts()

Male      23424
Female    17835
Name: Gender, dtype: int64

In [21]:
train_encode = {'Gender': {"Male":0, 'Female':1}, 'Geography': {'France':0, 'Spain':1, 'Germany':2}}
test_encode = {'Gender': {"Male":0, 'Female':1}, 'Geography': {'France':0, 'Spain':1, 'Germany':2}}

x_train = x_train.replace(train_encode)
x_test = x_test.replace(test_encode)

In [22]:
x_train.head()

,Unnamed: 0,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
23629,23629,646.0,1,1,40.0,8,171740.69,1,1,0,121592.50
6996,6996,676.0,0,1,21.0,10,79340.95,1,0,1,161579.19
26510,26510,710.0,0,1,53.0,8,0.00,1,0,0,168816.22
40020,40020,613.0,2,0,45.0,7,104796.54,2,1,0,80178.54
17668,17668,589.0,1,0,63.0,3,0.00,2,1,1,77864.41


**7. Prediction Model**

In [23]:
# Random Forest

rf_class = RandomForestClassifier()
rf_class.fit(x_train, y_train)

y_pred1 = rf_class.predict(x_test)

print('Random Forest Classification Report\n')
print(classification_report(y_test, y_pred1, target_names = ['0', '1']))

Random Forest Classification Report

              precision    recall  f1-score   support

           0       0.88      0.95      0.91      6486
           1       0.74      0.53      0.62      1766

    accuracy                           0.86      8252
   macro avg       0.81      0.74      0.77      8252
weighted avg       0.85      0.86      0.85      8252



In [24]:
# randon forest tuning parameter
parameters = {
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_depth': [6, 8, 10, 15]
}


rf_class2 = RandomForestClassifier()
rf_class2= GridSearchCV(rf_class2, param_grid = parameters, scoring='accuracy', cv=5)  

In [25]:
rf_class2.fit(x_train,y_train)
print("Tuned Hyperparameters :", rf_class2.best_params_)

Tuned Hyperparameters : {'criterion': 'entropy', 'max_depth': 10}


In [26]:
# best parameter random forest
rf_best = RandomForestClassifier(criterion= 'gini', max_depth= 10)
rf_best.fit(x_train, y_train)

y_predBest = rf_best.predict(x_test)

print('Best Random Forest Classification Report\n')
print(classification_report(y_test, y_predBest, target_names = ['0', '1']))

Best Random Forest Classification Report

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      6486
           1       0.77      0.52      0.62      1766

    accuracy                           0.86      8252
   macro avg       0.82      0.74      0.77      8252
weighted avg       0.86      0.86      0.85      8252



In [27]:
# xgboost classification
xgb_class = XGBClassifier()
xgb_class.fit(x_train, y_train)

y_pred2 = xgb_class.predict(x_test)

print('Xgboost Classification Report\n')
print(classification_report(y_test, y_pred2, target_names = ['0', '1']))

Xgboost Classification Report

              precision    recall  f1-score   support

           0       0.89      0.94      0.91      6486
           1       0.73      0.55      0.63      1766

    accuracy                           0.86      8252
   macro avg       0.81      0.75      0.77      8252
weighted avg       0.85      0.86      0.85      8252



model yang akan digunakan adalah random forest setelah tunning